In [1]:
import os
import pandas as pd
import numpy as np
import scipy.io


class BLP(object):
    
    def __init__(self, path, file):
        data = scipy.io.loadmat(os.path.join(path, file))
        self.data = data
        v_list = ['outshr', 'const', 'mpd', 'air', 'space', 'hpwt', 'price', 'trend']
        self.df = data['share']
        for item in v_list:
            self.df = np.concatenate([self.df, data[item]], axis=1)
        self.df = pd.DataFrame(self.df, columns = ['share'] + v_list)
        
        self.attributes = ['const', 'mpd', 'air', 'space', 'hpwt', 'price']
        
        '''
        creat instruments
        '''
        
    def ols(self):
        '''
        replicate the first column of table 3
        '''
        y = np.log(df['share']/df['outshr'])
        xmat = self.df[self.attributes].to_numpy()
        
    def iv(self):
        pass
    
    def market_share(self, para):
        pass
    
    def fixed_point(self,para):
        pass
    
       
                
if __name__ == "__main__":
    blp = BLP("/kaggle/input/blp-data/", "BLP_data.mat")


In [3]:
d0 = blp.df[blp.df['trend'] == 13]
blp.df.head()

,share,outshr,const,mpd,air,space,hpwt,price,trend
0,0.001051,0.880106,1.0,1.888146,0.0,1.1502,0.528997,4.935802,0.0
1,0.000670,0.880106,1.0,1.935989,0.0,1.2780,0.494324,5.516049,0.0
2,0.000341,0.880106,1.0,1.716799,0.0,1.4592,0.467613,7.108642,0.0
3,0.000522,0.880106,1.0,1.687871,0.0,1.6068,0.426540,6.839506,0.0
4,0.000442,0.880106,1.0,1.504286,0.0,1.6458,0.452489,8.928395,0.0


In [ ]:
df0 = blp.df[df['trend']==0]

In [17]:
x = np.concatenate([blp.data['price'], blp.data['air']], axis=1)
x.shape

(2217, 2)